# **SOM** - Grid map & Random initialization of weights

## Import libraries

In [1]:
import numpy as np
from keras.datasets import mnist
from matplotlib import pyplot as plt
import time
from math import exp


## Load the dataset

In [2]:
iterations = 20
neighbourhood_radius = 1

(_, _), (test_images, test_labels) = mnist.load_data()

reduced_mnist_images = [test_images[np.random.choice(a=np.where(test_labels == j)[0], size=100)] for j in range(10)]
reduced_mnist_images = np.array(reduced_mnist_images, dtype='float32') / 255

reduced_mnist_images = reduced_mnist_images.reshape((1000, 784))
reduced_mnist_labels = np.array([100*[i] for i in range(10)], dtype='int').ravel()

shuffled_indices = np.random.permutation(np.arange(1000))

reduced_mnist_images = reduced_mnist_images[shuffled_indices]
reduced_mnist_labels = reduced_mnist_labels[shuffled_indices]

## Implementation

In [3]:
def learning_rate_generator_with_decay():
   global iterations
   time = 0
   while True:
       yield (1 - (time/iterations))
       time += 1

In [4]:
# Define and fill the weights of grid network with random numbers (between 0 and 1)

grid_som_weight_matrix = np.random.rand(25, 25, 784) / 10

In [5]:
lr_with_decay = learning_rate_generator_with_decay()    # instantiate a lr generator
weights_snapshot = {0 : grid_som_weight_matrix.copy()}   # snapshot for some epochs
coverage_history = np.zeros((iterations+1, 25, 25), dtype=bool)

winning_counts = dict()
winning_classes_counts = dict()

for epoch in range(iterations):

    lr = next(lr_with_decay)    # Get new learning rate value

    start = time.time()     # start the timer

    for x in range(1000):

        # Calculate Euclidean distance for all neurons
        distance_matrix = np.sqrt(((grid_som_weight_matrix - reduced_mnist_images[x])**2).sum(axis = 2))

        # Find index of minimum distance
        winner_unit = np.argmin(distance_matrix)
        d0_winner = winner_unit // 25
        d1_winner = winner_unit % 25
        coverage_history[epoch, d0_winner, d1_winner] = True


        for d0 in range(d0_winner - 1, d0_winner + 1 + 1):
            for d1 in range(d1_winner - 1, d1_winner + 1 + 1):
                if 0 <= d0 <= 24 and 0 <= d1 <= 24:
                    if (d0 == d0_winner and d1 == d1_winner) or neighbourhood_radius == 1:
                        dif = reduced_mnist_images[x] - grid_som_weight_matrix[d0][d1]
                        grid_som_weight_matrix[d0][d1] += (dif * lr)

        # Decrease the neighbourhood radius if we are in the middle of running
        if round(iterations/3) == epoch:
            neighbourhood_radius = 0
        
        # If it is the last epoch, we want to store the winning statistics of neurons
        if epoch + 1 == iterations:
            winning_counts[(d0_winner,d1_winner)] = winning_counts.get((d0_winner,d1_winner), 0) + 1
            if winning_classes_counts.get((d0_winner,d1_winner), None) == None: winning_classes_counts[(d0_winner,d1_winner)] = dict()
            winning_classes_counts[(d0_winner,d1_winner)][reduced_mnist_labels[x]] = winning_classes_counts[(d0_winner,d1_winner)].get(reduced_mnist_labels[x], 0) + 1
    
    end = time.time()
    print("epoch %d - %.3f s" % (epoch + 1, end - start))

    weights_snapshot[epoch + 1] = grid_som_weight_matrix.copy()


epoch 1 - 1.361 s
epoch 2 - 1.390 s
epoch 3 - 1.214 s
epoch 4 - 1.268 s
epoch 5 - 1.287 s
epoch 6 - 1.319 s
epoch 7 - 1.315 s
epoch 8 - 1.353 s
epoch 9 - 1.488 s
epoch 10 - 1.530 s
epoch 11 - 1.561 s
epoch 12 - 1.561 s
epoch 13 - 1.552 s
epoch 14 - 1.489 s
epoch 15 - 1.546 s
epoch 16 - 1.558 s
epoch 17 - 1.554 s
epoch 18 - 1.606 s
epoch 19 - 1.593 s
epoch 20 - 1.591 s


### Show twenty neurons with the most winnings at the last epoch

In [6]:
twenty_neurons_with_most_winning = [k for k, v in sorted(winning_counts.items(), key=lambda item: item[1], reverse = True)][:20]

stat_table = np.zeros((20,10), dtype='int')

for i, neu in enumerate(twenty_neurons_with_most_winning):
    for class_label in winning_classes_counts[neu].keys():
        stat_table[i][class_label] = winning_classes_counts[neu][class_label]

import pandas as pd 

df = pd.DataFrame(stat_table, index=twenty_neurons_with_most_winning, columns=['label {0}'.format(i) for i in range(0,10)])
df['Total'] = df.sum(axis=1)
print("Total sum:  %d images" % df['Total'].sum())
df

Total sum:  138 images


,label 0,label 1,label 2,label 3,label 4,label 5,label 6,label 7,label 8,label 9,Total
"(24, 14)",0,0,0,0,0,0,0,11,0,0,11
"(2, 10)",0,10,0,0,0,0,0,0,0,0,10
"(1, 10)",0,10,0,0,0,0,0,0,0,0,10
"(3, 8)",0,7,0,0,0,0,0,0,0,1,8
"(10, 4)",0,0,0,0,0,0,8,0,0,0,8
"(5, 12)",0,7,0,0,0,0,0,0,0,0,7
"(21, 19)",0,0,0,0,7,0,0,0,0,0,7
"(1, 11)",0,6,0,0,0,0,0,0,0,1,7
"(13, 19)",0,0,0,0,0,7,0,0,0,0,7
"(20, 15)",0,0,0,0,0,0,0,0,0,6,6


### Generate and save figures

In [7]:
import os
os.makedirs('./grid-topology_random-weights/', exist_ok=True)

In [8]:
for epoch, weight_mat in weights_snapshot.items():
    fig = plt.figure(figsize=(15,15))
    axes = [fig.add_subplot(25,25,i+1) for i in range(625)]

    for n, ax in enumerate(axes):
        d0 = n // 25
        d1 = n % 25

        ax.imshow(weight_mat[d0,d1].reshape((28,28)), cmap=("gray_r" if coverage_history[epoch, d0, d1] == False else "plasma"))

        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        if n >= 600: ax.set_xlabel(n%25 + 1)
        if n%25==0: ax.set_ylabel(n//25 + 1)
        ax.tick_params(axis='both', which='both', length=0)
        ax.label_outer

    fig.subplots_adjust(wspace=0, hspace=0)
    fig.suptitle("Epoch %d - [Grid Topology , Random Initialization]" % epoch, fontsize=18)

    # write the figure for this epoch on disk
    path = f"./grid-topology_random-weights/epoch-{epoch}"
    fig.savefig(path)
    plt.close(fig)

    print(f"Saved: {path}.png")


Saved: ./grid-topology_random-weights/epoch-0.png
Saved: ./grid-topology_random-weights/epoch-1.png
Saved: ./grid-topology_random-weights/epoch-2.png
Saved: ./grid-topology_random-weights/epoch-3.png
Saved: ./grid-topology_random-weights/epoch-4.png
Saved: ./grid-topology_random-weights/epoch-5.png
Saved: ./grid-topology_random-weights/epoch-6.png
Saved: ./grid-topology_random-weights/epoch-7.png
Saved: ./grid-topology_random-weights/epoch-8.png
Saved: ./grid-topology_random-weights/epoch-9.png
Saved: ./grid-topology_random-weights/epoch-10.png
Saved: ./grid-topology_random-weights/epoch-11.png
Saved: ./grid-topology_random-weights/epoch-12.png
Saved: ./grid-topology_random-weights/epoch-13.png
Saved: ./grid-topology_random-weights/epoch-14.png
Saved: ./grid-topology_random-weights/epoch-15.png
Saved: ./grid-topology_random-weights/epoch-16.png
Saved: ./grid-topology_random-weights/epoch-17.png
Saved: ./grid-topology_random-weights/epoch-18.png
Saved: ./grid-topology_random-weights/epo

In [9]:
!zip -r grid_random.zip /content/grid-topology_random-weights

  adding: content/grid-topology_random-weights/ (stored 0%)
  adding: content/grid-topology_random-weights/epoch-20.png (deflated 10%)
  adding: content/grid-topology_random-weights/epoch-14.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-13.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-3.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-5.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-4.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-10.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-6.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-2.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-0.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-18.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-11.png (deflated 1%)
  adding: content/grid-topology_random-weights/epoch-1.pn